In [1]:
import numpy as np
import pandas as pd
import random
import itertools

In [2]:
a = []
# b = []
matrix = []
heatmapMatrix = []

locList = ["Grocery Stores, Supermarkets", "Bakeries", "Fast Food Restaurants", "Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances", "Drug Stores and Pharmacies", "Book Stores", "Motor Freight Carriers, Moving and Storage Companies, Trucking – Local/Long Distance, Delivery Services – Local"]

for j in range(len(locList)):
    prob = 1
    a = []
    b = []
    for i in range(len(locList)):
        if i != len(locList)-1:
            num = 0.16 * (i+1) *  np.random.rand()
            indprob = num * prob
            a.append(indprob)
            heatmapMatrix.append([locList[i], locList[j], indprob])
            prob = prob - indprob
        else:
            a.append(prob)
            heatmapMatrix.append([locList[i], locList[j], prob])
    matrix.append(a)
    # heatmapMatrix.append(b)

# print(heatmapMatrix)
probMatrix = pd.DataFrame(matrix, columns = locList, index = locList)
heatmapProbMatrix = pd.DataFrame(heatmapMatrix, columns=["place1", "place2", "prob"])

In [3]:
probMatrix.head()

,"Grocery Stores, Supermarkets",Bakeries,Fast Food Restaurants,"Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances",Drug Stores and Pharmacies,Book Stores,"Motor Freight Carriers, Moving and Storage Companies, Trucking – Local/Long Distance, Delivery Services – Local"
"Grocery Stores, Supermarkets",0.123903,0.024697,0.110073,0.457358,0.207044,0.029884,0.047041
Bakeries,0.071538,0.170701,0.098197,0.275974,0.206288,0.064683,0.112618
Fast Food Restaurants,0.109720,0.179229,0.106288,0.103282,0.170998,0.167724,0.162759
"Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances",0.037219,0.015807,0.064424,0.329816,0.188175,0.225714,0.138845
Drug Stores and Pharmacies,0.149531,0.171125,0.312725,0.063247,0.082998,0.078344,0.142030


In [4]:
pd.Timestamp("2020-04-30 09:00:00") + pd.offsets.Minute(10)  

Timestamp('2020-04-30 09:10:00')

In [130]:
allRec = []
routeLenth = 11
start_time = pd.Timestamp("2020-04-30")

# for j in range(routeLenth-1):
# #     rec = []
#     name = "people"+str(j)
pplCount = 0
for j in range(100):
    for num, location in zip(range(len(locList)), locList):
        name = "people"+str(pplCount)
        station = location
        rec = []
        time = pd.Timestamp("2020-04-30 09:00:00") 
        rec.append(name)#name
        rec.append(station)#location
        rec.append(time) #time
        rec.append(np.random.randint(10, high=80)) #transaction val
        allRec.append(rec)
        pplCount += 1
        for i in range(routeLenth-1):
            rec = []
            time += pd.offsets.Minute(np.random.randint(1, high=200))  
            # print(station)
            probList = probMatrix.loc[station].to_list()
            station = np.random.choice(locList, 1, p=probList)[0]
            rec.append(name)#name
            rec.append(station)#location
            rec.append(time) #time
            rec.append(np.random.randint(10, high=80)) #transaction val
            allRec.append(rec)



In [131]:
records = pd.DataFrame(allRec, columns = ["name", "mcc", "time", "transaction_value"])

In [132]:
records.head()

,name,mcc,time,transaction_value
0,people0,"Grocery Stores, Supermarkets",2020-04-30 09:00:00,47
1,people0,Fast Food Restaurants,2020-04-30 10:42:00,62
2,people0,"Furniture, Home Furnishings, and Equipment Sto...",2020-04-30 13:44:00,33
3,people0,Fast Food Restaurants,2020-04-30 15:11:00,75
4,people0,Fast Food Restaurants,2020-04-30 18:08:00,72


In [133]:
len(records)

7700

In [134]:
pd.options.mode.chained_assignment = None

In [135]:
nameList = records["name"].unique()
record_with_tinterval = pd.DataFrame([], columns = ["name", "mcc", "time", "transaction_value", "time_interval_to_next", "time_interval_from_last"])
#calculate time interval
for name in nameList:
    record_by_name = records[records["name"] == name]
    record_by_name["time_interval_to_next"] = -record_by_name["time"].diff(periods = -1)
    record_by_name["time_interval_from_last"] = record_by_name["time"].diff(periods = 1)
    record_by_name.replace(pd.Timedelta('nat'), pd.Timedelta(0), inplace=True)
    
    record_with_tinterval = record_with_tinterval.append(record_by_name)
#     print(record_by_name.head(2), record_with_tinterval.head())

record_with_tinterval.head()

,name,mcc,time,transaction_value,time_interval_to_next,time_interval_from_last
0,people0,"Grocery Stores, Supermarkets",2020-04-30 09:00:00,47,01:42:00,00:00:00
1,people0,Fast Food Restaurants,2020-04-30 10:42:00,62,03:02:00,01:42:00
2,people0,"Furniture, Home Furnishings, and Equipment Sto...",2020-04-30 13:44:00,33,01:27:00,03:02:00
3,people0,Fast Food Restaurants,2020-04-30 15:11:00,75,02:57:00,01:27:00
4,people0,Fast Food Restaurants,2020-04-30 18:08:00,72,00:07:00,02:57:00


In [136]:
#apply time filter: start time -> end time
startTime = pd.Timestamp("2020-04-30 10:00:00")
endTime = pd.Timestamp("2020-04-30 18:00:00")
test_time_range_filtered = record_with_tinterval[(record_with_tinterval["time"] > startTime)
                      & (record_with_tinterval["time"] > endTime)]

In [137]:
midTime = startTime + (endTime - startTime)/2
midTime

Timestamp('2020-04-30 14:00:00')

In [138]:
test_time_range_filtered.head()

,name,mcc,time,transaction_value,time_interval_to_next,time_interval_from_last
4,people0,Fast Food Restaurants,2020-04-30 18:08:00,72,00:07:00,02:57:00
5,people0,Drug Stores and Pharmacies,2020-04-30 18:15:00,30,01:06:00,00:07:00
6,people0,"Grocery Stores, Supermarkets",2020-04-30 19:21:00,42,02:09:00,01:06:00
7,people0,"Furniture, Home Furnishings, and Equipment Sto...",2020-04-30 21:30:00,11,00:04:00,02:09:00
8,people0,Fast Food Restaurants,2020-04-30 21:34:00,17,02:32:00,00:04:00


In [141]:
%%time
test_time_range_filtered.groupby("name")["name"].value_counts().max()

CPU times: user 3.58 ms, sys: 1.1 ms, total: 4.68 ms
Wall time: 3.62 ms


8

In [172]:
len(test_time_range_filtered)

3489

In [140]:
%%time
columns = ["mcc", "transaction_value", "time_interval_to_next", "time_interval_from_last"]
headr_initial = list(itertools.product([0], columns))
cols_initial = pd.MultiIndex.from_tuples(headr_initial)
routes_converted = pd.DataFrame([], columns = cols_initial)
for people in test_time_range_filtered["name"].unique():
#     print(people)
    stepCounts = 0
    for rows in test_time_range_filtered.loc[test_time_range_filtered["name"] == people, columns].itertuples(index=False):
        headr = list(itertools.product([stepCounts], columns))
        # print(headr)
        cols = pd.MultiIndex.from_tuples(headr)
        if stepCounts == 0:
            this_row = pd.DataFrame([rows], columns = cols)
        else:
            this_row = this_row.join(pd.DataFrame([rows], columns = cols), how = "outer")
#         print(this_row)
        stepCounts += 1
    this_row["name"] = people
    routes_converted = routes_converted.append(this_row, ignore_index=True)
# pd.DataFrame([rows], columns = cols)
# routes_converted.set_index("name").reset_index()


CPU times: user 16.2 s, sys: 27 ms, total: 16.2 s
Wall time: 16.2 s


In [176]:
%%time
columns = ["mcc", "transaction_value", "time_interval_to_next", "time_interval_from_last"]
headr = list(itertools.product(range(test_time_range_filtered.groupby("name")["name"].value_counts().max()), columns))
cols = pd.MultiIndex.from_tuples(headr)
all_rows = []

for people in test_time_range_filtered["name"].unique():
    this_row = []
    for rows in test_time_range_filtered.loc[test_time_range_filtered["name"] == people, columns].itertuples(index=False):
        for elements in rows:
            this_row.append(elements)
    all_rows.append(this_row)

all_routes_converted = pd.DataFrame(all_rows, columns = cols)
all_routes_converted["name"] = test_time_range_filtered["name"].unique()

CPU times: user 1.06 s, sys: 2.17 ms, total: 1.06 s
Wall time: 1.06 s


In [168]:
all_rows_df

0                    \
                                                   mcc transaction_value   
0                                Fast Food Restaurants                72   
1    Furniture, Home Furnishings, and Equipment Sto...                51   
2    Furniture, Home Furnishings, and Equipment Sto...                67   
3                                Fast Food Restaurants                67   
4                                Fast Food Restaurants                29   
..                                                 ...               ...   
694                         Drug Stores and Pharmacies                62   
695  Motor Freight Carriers, Moving and Storage Com...                32   
696  Furniture, Home Furnishings, and Equipment Sto...                48   
697                         Drug Stores and Pharmacies                33   
698                                           Bakeries                74   

                                                   \
    time_interval_to_next time_interval_from_last   
0                00:07:00                02:57:00   
1                03:07:00                03:03:00   
2                00:02:00                01:35:00   
3                00:17:00                01:49:00   
4                00:11:00                02:35:00   
..                    ...                     ...   
694              01:00:00                00:31:00   
695              00:16:00                02:12:00   
696              00:02:00                03:19:00   
697              02:14:00                01:32:00   
698              02:45:00                01:18:00   

                                                     1                    \
                                                   mcc transaction_value   
0                           Drug Stores and Pharmacies              30.0   
1    Motor Freight Carriers, Moving and Storage Com...              37.0   
2    Furniture, Home Furnishings, and Equipment Sto...              11.0   
3    Furniture, Home Furnishings, and Equipment Sto...              49.0   
4    Motor Freight Carriers, Moving and Storage Com...              78.0   
..                                                 ...               ...   
694                       Grocery Stores, Supermarkets              14.0   
695                                        Book Stores              30.0   
696                         Drug Stores and Pharmacies              73.0   
697                         Drug Stores and Pharmacies              38.0   
698  Furniture, Home Furnishings, and Equipment Sto...              27.0   

                                                   \
    time_interval_to_next time_interval_from_last   
0                01:06:00                00:07:00   
1                00:06:00                03:07:00   
2                00:39:00                00:02:00   
3                00:57:00                00:17:00   
4                00:41:00                00:11:00   
..                    ...                     ...   
694              00:45:00                01:00:00   
695              00:02:00                00:16:00   
696              02:41:00                00:02:00   
697              03:04:00                02:14:00   
698              00:11:00                02:45:00   

                                                     2                    ...  \
                                                   mcc transaction_value  ...   
0                         Grocery Stores, Supermarkets              42.0  ...   
1                                Fast Food Restaurants              11.0  ...   
2                                          Book Stores              13.0  ...   
3    Furniture, Home Furnishings, and Equipment Sto...              66.0  ...   
4                                          Book Stores              46.0  ...   
..                                                 ...               ...  ...   
694                         Drug Stores and Pha

In [155]:
routes_converted.set_index("name").reset_index()

name                                                  0  \
                                                              mcc   
0      people0                              Fast Food Restaurants   
1      people1  Furniture, Home Furnishings, and Equipment Sto...   
2      people2  Furniture, Home Furnishings, and Equipment Sto...   
3      people3                              Fast Food Restaurants   
4      people4                              Fast Food Restaurants   
..         ...                                                ...   
694  people695                         Drug Stores and Pharmacies   
695  people696  Motor Freight Carriers, Moving and Storage Com...   
696  people697  Furniture, Home Furnishings, and Equipment Sto...   
697  people698                         Drug Stores and Pharmacies   
698  people699                                           Bakeries   

                                                                     \
    time_interval_from_last time_interval_to_next transaction_value   
0                  02:57:00              00:07:00                72   
1                  03:03:00              03:07:00                51   
2                  01:35:00              00:02:00                67   
3                  01:49:00              00:17:00                67   
4                  02:35:00              00:11:00                29   
..                      ...                   ...               ...   
694                00:31:00              01:00:00                62   
695                02:12:00              00:16:00                32   
696                03:19:00              00:02:00                48   
697                01:32:00              02:14:00                33   
698                01:18:00              02:45:00                74   

                                                     1  \
                                                   mcc   
0                           Drug Stores and Pharmacies   
1    Motor Freight Carriers, Moving and Storage Com...   
2    Furniture, Home Furnishings, and Equipment Sto...   
3    Furniture, Home Furnishings, and Equipment Sto...   
4    Motor Freight Carriers, Moving and Storage Com...   
..                                                 ...   
694                       Grocery Stores, Supermarkets   
695                                        Book Stores   
696                         Drug Stores and Pharmacies   
697                         Drug Stores and Pharmacies   
698  Furniture, Home Furnishings, and Equipment Sto...   

                                                                     \
    time_interval_from_last time_interval_to_next transaction_value   
0                  00:07:00              01:06:00              30.0   
1                  03:07:00              00:06:00              37.0   
2                  00:02:00              00:39:00              11.0   
3                  00:17:00              00:57:00              49.0   
4                  00:11:00              00:41:00              78.0   
..                      ...                   ...               ...   
694                01:00:00              00:45:00              14.0   
695                00:16:00              00:02:00              30.0   
696                00:02:00              02:41:00              73.0   
697                02:14:00              03:04:00              38.0   
698                02:45:00              00:11:00              27.0   

                                                     2  ...  \
                                                   mcc  ...   
0                         Grocery Stores, Supermarkets  ...   
1                                Fast Food Restaurants  ...   
2                                          Book Stores  ...   
3    Furniture, Home Furnishings, and Equipment Sto...  ...   
4                                          Book Stores  ...   
..                                                 ...  ...   
694  